# Usage Examples

This document outlines the different functionalities available within the Box Embeddings package

### 0. Installing the package on your machine

*If you have the repo cloned*
```
pip install --editable . --user
```

### A. Initialize a box tensor and check its parameters

#### Standard Box Tensor
To represent a Tensor as a Box, we use the class `BoxTensor`. The necessary parameter is `data` (a tensor)

In [59]:
from box_embeddings.parameterizations.box_tensor import *

# Let's create a toy example
data = torch.tensor([[[1, 2, 3], [3, 4, 6]],
          [[5, 6, 8], [7, 9, 5]]])
print(data.size())
box = BoxTensor(data)

torch.Size([2, 2, 3])


We can use several methods to look at the parameters of our box, such as

In [60]:
# Lower left coordinate
print(box.z)
# Top right coordinate
print(box.Z)
# Center coordinate
print(box.centre)


tensor([[1, 2, 3],
        [5, 6, 8]])
tensor([[3, 4, 6],
        [7, 9, 5]])
tensor([[2.0000, 3.0000, 4.5000],
        [6.0000, 7.5000, 6.5000]])


Let's broadcast our box to a new shape.. Broadcasting is often needed for different arithmetic operations. The function we
will use is `broadcast()`, and the required parameter is `target_shape=()`, which specify the new shape
for the box. This is very similar to `numpy.broadcast_to()`

In [74]:
data = torch.tensor([[[1, 2, 3], [3, 4, 6]],
          [[5, 6, 8], [7, 9, 5]]])
box = BoxTensor(data)
print('previous shape is:', box.box_shape)
box.broadcast(target_shape=(1, 2, 3))
print('after broadcasting:', box.box_shape)

previous shape is: (2, 3)
after broadcasting: (1, 2, 3)


#### Delta Box Tensor
We can initialize a box with additional requirement, such as forcing `Z=z + delta` using the method `MinDeltaBoxTensor`

In [42]:
from box_embeddings.parameterizations.delta_box_tensor import *
data = torch.tensor([[1, 2, 3], [3, 4, 6]])
delta_box = MinDeltaBoxTensor(data)
delta_box.z
delta_box.beta

1.0

#### Sigmoid Box Tensor

In [2]:
# TODO
from box_embeddings.parameterizations.sigmoid_box_tensor import *

ImportError: cannot import name 'inv_sigmoid' from 'box_embeddings.common.utils' (/Users/mm12574/box-embeddings/box_embeddings/common/utils.py)

#### Tanh Box Tensor

In [4]:
# TODO
from box_embeddings.parameterizations.tanh_box_tensor import *

#### Uniform Box
Using the function `uniform_boxes`, we can create several uniform boxes specified by `num_boxes` such that each of
them is inside the bounding box defined by `(minimum,maximum)` in each dimension. The defaults are `minimum=0.0`
and `maximum=1.0`. We can also specify `(delta_min, delta_max)`. The defaults are `delta_min=0.01` and `delta_max=0.5`

In [23]:
# Initialize
from box_embeddings.initializations.uniform_boxes import *
uniform = uniform_boxes(dimensions = 2, num_boxes = 3, minimum = 1.0, maximum = 4.0)

This function returns a tuple of `(z, Z)`, where `z` and `Z` are tensors. `z` are the lower left coordinates of
the 3 boxes in this example. Similarly, `Z` are the top right coordinates.

In [25]:
# Return (z,Z)
uniform

(tensor([[2.1434, 3.3852],
         [1.9863, 2.2522],
         [1.3006, 3.7267]], dtype=torch.float64),
 tensor([[2.4814, 3.4371],
         [2.0022, 2.2938],
         [1.5343, 3.7707]], dtype=torch.float64))

### 2. Box Volume

In [75]:
# TODO
# Currently no codes yet
from box_embeddings.modules.volume import *

### 3. Box Regularization

In [76]:
# TODO
# Currently no codes yet
from box_embeddings.modules.regularization import *

### 4. Box Intersection

In [1]:
from box_embeddings.modules.intersection import *

AllenNLP not available. Registrable won't work.


### 5. Box Pooling

In [78]:
from box_embeddings.modules.pooling import *
